In [2]:
#Importing libraries

library(tidyverse)
library(dplyr)
library(lubridate)
library(tidyr)
library(Hmisc)
library(gapminder)

In [3]:
#loading data set
eu_storedata <- readxl::read_xls("C:/Users/DELL/Downloads/Sample - EU Superstore.xls")


In [31]:
#Loading data set of customer metrics
Final_submissions <- read.csv("C:/Users/DELL/Desktop/finalsubmission/Final_submissions.csv")

In [4]:
#saving date that will be used as current date("Checkdate")
Checkdate <-  as.POSIXct("2019-01-31")


In [5]:
#creating new orderID which is unique for each customer
df_eu_storedata <- eu_storedata |>  mutate(Order_ID = paste(`Order ID`, `Customer ID`, sep = '_'))


# SALES GROWTH RATE FOR EACH YEAR


In [14]:
# Calculate sales growth rate for 2016
  sales_growth_rate_2016 <- (sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2016-01-01' & df_eu_storedata$`Order Date` <= '2016-12-31']) - 
                               sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2015-01-01' & df_eu_storedata$`Order Date` <= '2015-12-31'])) / 
    sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2015-01-01' & df_eu_storedata$`Order Date` <= '2015-12-31']) * 100
  
  # Calculate sales growth rate for 2017
  sales_growth_rate_2017 <- (sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2017-01-01' & df_eu_storedata$`Order Date` <= '2017-12-31']) - 
                               sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2016-01-01' & df_eu_storedata$`Order Date` <= '2016-12-31'])) / 
    sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2016-01-01' & df_eu_storedata$`Order Date` <= '2016-12-31']) * 100
  
  # Calculate sales growth rate for 2018
  sales_growth_rate_2018 <- (sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2018-01-01' & df_eu_storedata$`Order Date` <= '2018-12-31']) - 
                               sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2017-01-01' & df_eu_storedata$`Order Date` <= '2017-12-31'])) / 
    sum(df_eu_storedata$Sales[df_eu_storedata$`Order Date` >= '2017-01-01' & df_eu_storedata$`Order Date` <= '2017-12-31']) * 100
  
print(sales_growth_rate_2016)
print(sales_growth_rate_2017)
print(sales_growth_rate_2018)

[1] 36.84067
[1] 16.22155
[1] 37.05488


# Calculate churn rate

In [17]:
churn_rate <-
  df_eu_storedata |>
    group_by(`Customer ID`) |>
    summarise(last_purchase_date = max(`Order Date`)) |>
    mutate(churned = ifelse(last_purchase_date < "2018-01-01", 1, 0)) |>
    summarise(churn_rate = (sum(churned)/n()) * 100)
print(churn_rate)

# A tibble: 1 × 1
  churn_rate
       <dbl>
1       11.2


# Finding New Customer Revenue Retention (NCRR)

In [22]:

  
  # Identify new customers who made their first purchase in 2017
  new_customers_2017 <- df_eu_storedata |>
    group_by(`Customer ID`) |>
    summarise(first_purchase_date = min(`Order Date`)) |>
    filter(first_purchase_date >= '2017-01-01' & first_purchase_date <= '2017-12-31') |>
    select(`Customer ID`) 
  
  # Identify customers who made purchases in both 2017 and 2018
  customers_2017_2018 <- df_eu_storedata |>
    group_by(`Customer ID`) |>
    summarise(total_sales_2017 = sum(Sales[`Order Date` >= '2017-01-01' & `Order Date` <= '2017-12-31']),
              total_sales_2018 = sum(Sales[`Order Date` >= '2018-01-01' & `Order Date` <= '2018-12-31'])) |> 
    filter(total_sales_2017 > 0 & total_sales_2018 > 0) |> 
    select(`Customer ID`) 
  
  # Calculate NCRR  
  NCRR <- (df_eu_storedata |>
             filter(`Customer ID` %in% customers_2017_2018$`Customer ID`) |>
             group_by(`Customer ID`) |>
             summarise(total_sales_2017 = sum(Sales[`Order Date` >= '2017-01-01' & `Order Date` <= '2017-12-31']),
                       total_sales_2018 = sum(Sales[`Order Date` >= '2018-01-01' & `Order Date` <= '2018-12-31']))) |>
    left_join(new_customers_2017, by = 'Customer ID') |> 
    summarise(revenue_from_new_customers_2017 = sum(total_sales_2017),
              revenue_lost_due_to_churn = sum(total_sales_2017[is.na(total_sales_2018)]),
              revenue_from_new_customers_2018 = sum(total_sales_2018[!is.na(total_sales_2018)])) |>
    mutate(NCRR = (revenue_from_new_customers_2018 - revenue_lost_due_to_churn) / revenue_from_new_customers_2017 * 100) |> 
    view()
  
print(NCRR)


# A tibble: 1 × 4
  revenue_from_new_customers_2017 revenue_lost_due_to_churn revenue_from…¹  NCRR
                            <dbl>                     <dbl>          <dbl> <dbl>
1                         677984.                         0        850279.  125.
# … with abbreviated variable name ¹​revenue_from_new_customers_2018


# Working for CLV

In [58]:
CLVs <- Final_submissions |>
  mutate(`Customer ID` = Customer.ID) |> 
  mutate(CV = mean(total_transactions) * mean(average_order_value)) |> 
  mutate(CLS = mean(difftime(as.Date(date_last_transaction, format = "%m/%d/%Y"), as.Date(date_first_transaction, format = "%m/%d/%Y"), units = "days") / 365)) |> 
  left_join(df_eu_storedata |>
              mutate(year = lubridate::year(`Order Date`)) |>
              group_by(`Customer ID`, year) |>
              summarise(year_finding = n_distinct(Order_ID)) |> 
              group_by(`Customer ID`) |>
              summarise(avg_order_per_year = mean(year_finding)), by = "Customer ID") |> 
  summarise(dataset_avg_order_per_year = mean(avg_order_per_year), 
            CLV = as.integer(CV * CLS * dataset_avg_order_per_year)) 

Customer_LV <- CLVs %>% 
     summarise(AVG_Order_Per_Year = max(dataset_avg_order_per_year),
              Customer_lifetime_value = max(CLV))

print(Customer_LV)

`summarise()` has grouped output by 'Customer ID'. You can override using the `.groups` argument.
Warning message:
"Returning more (or less) than 1 row per `summarise()` group was deprecated in dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()` always returns an ungrouped data frame
  and adjust accordingly."


  AVG_Order_Per_Year Customer_lifetime_value
1           2.058071                   20702


# Percentage of orders with high-priced products


In [29]:
Percentage_of_orders_with_high_priced_products <- df_eu_storedata |> 
  mutate(unit_price = Sales/Quantity) |> 
  mutate(Factors = cut2(unit_price)) |> 
  group_by(Factors) |> 
  summarise(orders = n_distinct(Order_ID)) |> 
  slice_max(Factors, n=1, with_ties = FALSE) |> 
   mutate(high_percentage = (orders/5121)*100,
          remainin_percentage = 100-((orders/5121)*100))

print(Percentage_of_orders_with_high_priced_products)

# A tibble: 1 × 4
  Factors         orders high_percentage remainin_percentage
  <fct>            <int>           <dbl>               <dbl>
1 [473.47,908.55]    148            2.89                97.1


# Finding percentage of customers who are regular and promotional


In [55]:
POC <- Final_submissions |> 
  group_by(shopper_type) |> 
  summarise(n_customer = n_distinct(`Customer.ID`)) |> 
  mutate(percentage = (n_customer/795)*100)

print(POC)

# A tibble: 2 × 3
  shopper_type        n_customer percentage
  <chr>                    <int>      <dbl>
1 Regular_shopper            528       66.4
2 promotional_shopper        267       33.6


# Finding average discount that customers get on their order


In [60]:
ADPPO <- df_eu_storedata |> 
  group_by(`Customer Name`,Order_ID) |> 
  summarise(OID_discount = sum(Discount)) |>  
  group_by(`Customer Name`) |> 
  summarise(Cust_disc = mean(OID_discount)*100) |> 
  mutate(discount_shopping = mean(Cust_disc),
         normal_shopping = (100-mean(Cust_disc)))

average_discount_percentage_per_order <- ADPPO %>% 
                                         summarise(Discount_shopping_percentage = max(discount_shopping),
                                                  Regular_shopping_percentage = max(normal_shopping))

print(average_discount_percentage_per_order)

`summarise()` has grouped output by 'Customer Name'. You can override using the `.groups` argument.


# A tibble: 1 × 2
  Discount_shopping_percentage Regular_shopping_percentage
                         <dbl>                       <dbl>
1                         20.1                        79.9


# Calculating profit percentage on store's total sales


In [62]:
Profit_percentage <- df_eu_storedata |>
    summarise(total_profit = sum(Profit), total_sales = sum(Sales)) |>
    mutate(profit_margin = (total_profit / total_sales) * 100)
  
print(Profit_percentage)

# A tibble: 1 × 3
  total_profit total_sales profit_margin
         <dbl>       <dbl>         <dbl>
1      372830.    2938089.          12.7
